In [1]:
import os
import sys
import IPython
os.environ['NUMBA_CACHE_DIR'] = IPython.paths.get_ipython_cache_dir()
sys.path.append(os.path.join(os.path.dirname(""), ".."))
import custom
import numpy as np

import librosa # 음향 데이터 전처리(melspectogram)
import json # 텍스트 데이터
import pickle # 전처리 데이터 저장, 소리 사전
from tqdm import tqdm

In [2]:
# 소리 사전 불러오기

with open("sound_dict.pkl", mode = "rb") as f:
    sound_dict = pickle.load(f)

print(len(sound_dict))
print(sound_dict['송'])
print(sound_dict['<eos>'])

11181
5538
11174


In [3]:


def preprocess(text_dir, sound_dir, text_save_dir, sound_save_dir, freq = 16000, n_mels = 64) :
    text_folder_dir_list = os.listdir(text_dir)
    
    # print(text_folder_dir_list)
    
    sound_list = []
    sound_len = []
    text_list = []
    text_len = []
    
    print("데이터 전처리")
    for folder in text_folder_dir_list : #소리 폴더 이름
        if folder.find(".") >= 0 : # 폴더가 아닌 것 (확장자 표시하는 . 이 존재하는 것)
            continue
        sound_folder = folder.replace("L","S") # 소리 폴더 이름
        sound_folder_dir = sound_dir + "/" + sound_folder # 소리 폴더 위치
        # print(sound_folder_dir)
        folder_dir = text_dir + "/" + folder # 텍스트 폴더 위치
        text_file_dir_list = os.listdir(folder_dir)
        # print(text_file_dir_list)
        for file in tqdm(text_file_dir_list) : #텍스트 파일 이름
            if file.find(".json") < 0 : #json 파일이 아닌 것
                continue
            file_dir = folder_dir + "/" + file #텍스트 파일 위치
            with open(file_dir, mode = "r", encoding="UTF8") as f :
                text_data = json.load(f)
            # print(text_data)
            text = text_data['Dialogs'][0]['SpeakerText']
            # print(text)
            text = list(text) # 텍스트를 한 글자 씩 나눈 것
            text.append('<eos>') 
            text = custom.word_vectorize(text, sound_dict)
            text_list.append(text)
            text_len.append(len(text))
            
            sound_file = text_data['MediaUrl'].split("/")[-1]
            sound_file_dir = sound_folder_dir + '/' + sound_file 
            
            signal, freq = librosa.load(sound_file_dir, sr = freq)
            S = librosa.feature.melspectrogram(y = signal, sr = freq, n_mels = n_mels) #멜스펙토그램 전처리
            S = S.transpose(1,0)
            # print(S.shape)
            sound_list.append(S)
            sound_len.append(len(S))
    
    max_text_len = max(text_len)
    max_sound_len = max(sound_len)
    
    print("텍스트 패딩")
    pad_idx = sound_dict['<pad>']
    for i in tqdm(range(len(text_list))) :
        for j in range(max_text_len - len(text_list[i])) :
            text_list[i].append(pad_idx)
    text_list = np.array(text_list)
    print("텍스트 데이터 형태 : ", text_list.shape)
    
    print("소리 패딩")
    for i in tqdm(range(len(sound_list))) :
        pad_len = max_sound_len - len(sound_list[i])
        if pad_len <= 0 :
            continue
        pad_array = np.zeros((pad_len,n_mels))
        sound_list[i] = np.append(sound_list[i], pad_array, axis = 0)
    sound_list = np.array(sound_list)
    print("소리 데이터 형태 : ", sound_list.shape)
    
    print("데이터 저장")
    with open(sound_save_dir, mode = "wb") as f:
        pickle.dump(sound_list, f)
    with open(text_save_dir, mode = "wb") as f:
        pickle.dump(text_list, f)    
    print("저장 완료")

In [4]:
os.makedirs('data/', exist_ok = True)

text_dir = "음성 자료/test/label"
sound_dir = "음성 자료/test/data"

preprocess(text_dir, sound_dir, "data/text_data_test.pkl", "data/sound_data_test.pkl")


text_dir = "음성 자료/train/label"
sound_dir = "음성 자료/train/data"

preprocess(text_dir, sound_dir, "data/text_data_train.pkl", "data/sound_data_train.pkl")

데이터 전처리


100%|████████████████████████████████████████████████████████████████████████████████| 232/232 [00:04<00:00, 55.83it/s]


텍스트 패딩


100%|██████████████████████████████████████████████████████████████████████████| 1250/1250 [00:00<00:00, 878940.49it/s]


텍스트 데이터 형태 :  (1250, 6)
소리 패딩


...

텍스트 패딩


100%|████████████████████████████████████████████████████████████████████████| 10002/10002 [00:00<00:00, 847691.98it/s]


텍스트 데이터 형태 :  (10002, 6)
소리 패딩


100%|██████████████████████████████████████████████████████████████████████████| 10002/10002 [00:01<00:00, 6330.86it/s]


소리 데이터 형태 :  (10002, 196, 64)
데이터 저장
저장 완료
